# Черновой вариант итогового проекта для курса "Python для Data Science"

In [2]:
import numpy as np
import pandas as pd
import pickle

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
%config InlineBackend.figure_format = 'svg'
pd.options.display.max_columns = 100

In [3]:
import warnings
warnings.filterwarnings('ignore')

### Описание датасета

1. Id - идентификационный номер квартиры
2. DistrictId - идентификационный номер района
3. Rooms - количество комнат
4. Square - площадь
5. LifeSquare - жилая площадь
6. KitchenSquare - площадь кухни
7. Floor - этаж
8. HouseFloor - количество этажей в доме
9. HouseYear - год постройки дома
10. Ecology_1, Ecology_2, Ecology_3 - экологические показатели местности
11. Social_1, Social_2, Social_3 - социальные показатели местности
12. Healthcare_1, Helthcare_2 - показатели местности, связанные с охраной здоровья
13. Shops_1, Shops_2 - показатели, связанные с наличием магазинов, торговых центров
14. Price - цена квартиры

### Загружаю датасеты

In [4]:
TRAIN_DATASET_PATH = 'datasets/train.csv'
TEST_DATASET_PATH = 'datasets/test.csv'
PREPARED_TRAIN_DATASET_PATH = 'datasets/train_prepared.csv'
PREPARED_TEST_DATASET_PATH = 'datasets/test_prepared.csv'
SCALED_TRAIN_DATASET_PATH = 'datasets/train_scaled.csv'
SCALED_TEST_DATASET_PATH = 'datasets/test_scaled.csv'
PREDICT_PRICE  = 'datasets/AVsasilev_predictions.csv'
X_TRAIN = 'datasets/X_train.csv'
X_VALID = 'datasets/X_valid.csv'
Y_TRAIN = 'datasets/y_train.csv'
Y_VALID = 'datasets/y_valid.csv'
SCALER_FILE_PATH = 'scaler.pkl'
MODEL_PATH = 'model.pkl'

##### Train dataset

In [5]:
df_train = pd.read_csv(TRAIN_DATASET_PATH)
df_train.head()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price
0,14038,35,2.0,47.981561,29.442751,6.0,7,9.0,1969,0.089040,B,B,33,7976,5,NaN,0,11,B,184966.930730
1,15053,41,3.0,65.683640,40.049543,8.0,7,9.0,1978,0.000070,B,B,46,10309,1,240.0,1,16,B,300009.450063
2,4765,53,2.0,44.947953,29.197612,0.0,8,12.0,1968,0.049637,B,B,34,7759,0,229.0,1,3,B,220925.908524
3,5809,58,2.0,53.352981,52.731512,9.0,8,17.0,1977,0.437885,B,B,23,5735,3,1084.0,0,5,B,175616.227217
4,10783,99,1.0,39.649192,23.776169,7.0,11,12.0,1976,0.012339,B,B,35,5776,1,2078.0,2,4,B,150226.531644


In [54]:
df_train.shape

(10000, 20)

In [55]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             10000 non-null  int64  
 1   DistrictId     10000 non-null  int64  
 2   Rooms          10000 non-null  float64
 3   Square         10000 non-null  float64
 4   LifeSquare     7887 non-null   float64
 5   KitchenSquare  10000 non-null  float64
 6   Floor          10000 non-null  int64  
 7   HouseFloor     10000 non-null  float64
 8   HouseYear      10000 non-null  int64  
 9   Ecology_1      10000 non-null  float64
 10  Ecology_2      10000 non-null  object 
 11  Ecology_3      10000 non-null  object 
 12  Social_1       10000 non-null  int64  
 13  Social_2       10000 non-null  int64  
 14  Social_3       10000 non-null  int64  
 15  Healthcare_1   5202 non-null   float64
 16  Helthcare_2    10000 non-null  int64  
 17  Shops_1        10000 non-null  int64  
 18  Shops_2

In [56]:
df_train.describe()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Price
count,10000.00000,10000.000000,10000.000000,10000.000000,7887.000000,10000.000000,10000.000000,10000.000000,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,5202.000000,10000.000000,10000.000000,10000.000000
mean,8383.40770,50.400800,1.890500,56.315775,37.199645,6.273300,8.526700,12.609400,3.990166e+03,0.118858,24.687000,5352.157400,8.039200,1142.904460,1.319500,4.231300,214138.857399
std,4859.01902,43.587592,0.839512,21.058732,86.241209,28.560917,5.241148,6.775974,2.005003e+05,0.119025,17.532614,4006.799803,23.831875,1021.517264,1.493601,4.806341,92872.293865
min,0.00000,0.000000,0.000000,1.136859,0.370619,0.000000,1.000000,0.000000,1.910000e+03,0.000000,0.000000,168.000000,0.000000,0.000000,0.000000,0.000000,59174.778028
25%,4169.50000,20.000000,1.000000,41.774881,22.769832,1.000000,4.000000,9.000000,1.974000e+03,0.017647,6.000000,1564.000000,0.000000,350.000000,0.000000,1.000000,153872.633942
50%,8394.50000,36.000000,2.000000,52.513310,32.781260,6.000000,7.000000,13.000000,1.977000e+03,0.075424,25.000000,5285.000000,2.000000,900.000000,1.000000,3.000000,192269.644879
75%,12592.50000,75.000000,2.000000,65.900625,45.128803,9.000000,12.000000,17.000000,2.001000e+03,0.195781,36.000000,7227.000000,5.000000,1548.000000,2.000000,6.000000,249135.462171
max,16798.00000,209.000000,19.000000,641.065193,7480.592129,2014.000000,42.000000,117.000000,2.005201e+07,0.521867,74.000000,19083.000000,141.000000,4849.000000,6.000000,23.000000,633233.466570


In [57]:
df_train.isnull().sum()

Id                  0
DistrictId          0
Rooms               0
Square              0
LifeSquare       2113
KitchenSquare       0
Floor               0
HouseFloor          0
HouseYear           0
Ecology_1           0
Ecology_2           0
Ecology_3           0
Social_1            0
Social_2            0
Social_3            0
Healthcare_1     4798
Helthcare_2         0
Shops_1             0
Shops_2             0
Price               0
dtype: int64

##### Test dataset

In [58]:
df_test = pd.read_csv(TEST_DATASET_PATH)
df_test.head()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2
0,725,58,2.0,49.882643,33.432782,6.0,6,14.0,1972,0.310199,B,B,11,2748,1,NaN,0,0,B
1,15856,74,2.0,69.263183,NaN,1.0,6,1.0,1977,0.075779,B,B,6,1437,3,NaN,0,2,B
2,5480,190,1.0,13.597819,15.948246,12.0,2,5.0,1909,0.000000,B,B,30,7538,87,4702.0,5,5,B
3,15664,47,2.0,73.046609,51.940842,9.0,22,22.0,2007,0.101872,B,B,23,4583,3,NaN,3,3,B
4,14275,27,1.0,47.527111,43.387569,1.0,17,17.0,2017,0.072158,B,B,2,629,1,NaN,0,0,A


In [59]:
df_test.shape

(5000, 19)

In [60]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             5000 non-null   int64  
 1   DistrictId     5000 non-null   int64  
 2   Rooms          5000 non-null   float64
 3   Square         5000 non-null   float64
 4   LifeSquare     3959 non-null   float64
 5   KitchenSquare  5000 non-null   float64
 6   Floor          5000 non-null   int64  
 7   HouseFloor     5000 non-null   float64
 8   HouseYear      5000 non-null   int64  
 9   Ecology_1      5000 non-null   float64
 10  Ecology_2      5000 non-null   object 
 11  Ecology_3      5000 non-null   object 
 12  Social_1       5000 non-null   int64  
 13  Social_2       5000 non-null   int64  
 14  Social_3       5000 non-null   int64  
 15  Healthcare_1   2623 non-null   float64
 16  Helthcare_2    5000 non-null   int64  
 17  Shops_1        5000 non-null   int64  
 18  Shops_2 

In [61]:
df_test.describe()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1
count,5000.000000,5000.000000,5000.000000,5000.000000,3959.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,2623.000000,5000.00000,5000.000000
mean,8412.595400,51.279200,1.910000,56.449500,36.158810,5.976800,8.632000,12.601000,1984.392600,0.119874,24.933800,5406.900000,8.262600,1146.657263,1.31940,4.242800
std,4832.674037,44.179466,0.838594,19.092787,17.825287,9.950018,5.483228,6.789213,18.573149,0.120070,17.532202,4026.614773,23.863762,1044.744231,1.47994,4.777365
min,1.000000,0.000000,0.000000,1.378543,0.333490,0.000000,1.000000,0.000000,1908.000000,0.000000,0.000000,168.000000,0.000000,0.000000,0.00000,0.000000
25%,4221.750000,21.000000,1.000000,41.906231,23.092026,1.000000,4.000000,9.000000,1973.000000,0.019509,6.000000,1564.000000,0.000000,325.000000,0.00000,1.000000
50%,8320.500000,37.000000,2.000000,52.921340,32.925087,6.000000,7.000000,12.000000,1977.000000,0.072158,25.000000,5285.000000,2.000000,900.000000,1.00000,3.000000
75%,12598.250000,77.000000,2.000000,66.285129,45.174091,9.000000,12.000000,17.000000,2000.000000,0.195781,36.000000,7287.000000,5.000000,1548.000000,2.00000,6.000000
max,16795.000000,212.000000,17.000000,223.453689,303.071094,620.000000,78.000000,99.000000,2020.000000,0.521867,74.000000,19083.000000,141.000000,4849.000000,6.00000,23.000000


In [62]:
df_test.isnull().sum()

Id                  0
DistrictId          0
Rooms               0
Square              0
LifeSquare       1041
KitchenSquare       0
Floor               0
HouseFloor          0
HouseYear           0
Ecology_1           0
Ecology_2           0
Ecology_3           0
Social_1            0
Social_2            0
Social_3            0
Healthcare_1     2377
Helthcare_2         0
Shops_1             0
Shops_2             0
dtype: int64

### Обработка пропусков

##### LifeSquare

In [63]:
lsquare_median_train = round(df_train["LifeSquare"].median(), 6)
lsquare_median_test = round(df_test["LifeSquare"].median(), 6)
print(f'Медиана LifeSquare обучающего датасета:\t {lsquare_median_train}')
print(f'Медиана LifeSquare тестового датасета:\t {lsquare_median_test}')

Медиана LifeSquare обучающего датасета:	 32.78126
Медиана LifeSquare тестового датасета:	 32.925087


Пропуски заполняю медианным значением LifeSquare, если это значение меньше значения Square

In [64]:
df_train.loc[df_train['LifeSquare'].isnull() & (df_train['Square'] > lsquare_median_train), 'LifeSquare'] = lsquare_median_train

In [65]:
df_test.loc[df_test['LifeSquare'].isnull() & (df_train['Square'] > lsquare_median_test), 'LifeSquare'] = lsquare_median_test

Оставшиеся пропуски заполняю разницей `Square` и `разницы медианных значений Square и LifeSquare`, если это значение меньше значения LifeSquare

In [66]:
sub_square_train = round(df_train['Square'].median() - df_train['LifeSquare'].median(), 6)
print(f'Разница медиан Square и LifeSquare обучающего датасета:\t {sub_square_train}')

Разница медиан Square и LifeSquare обучающего датасета:	 19.73205


In [67]:
df_train.loc[df_train['LifeSquare'].isnull() & (df_train['Square'] > sub_square_train), 'LifeSquare'] = df_train['Square'] - lsquare_median_train

In [68]:
sub_square_test = round(df_test['Square'].median() - df_test['LifeSquare'].median(), 6)
print(f'Разница медиан Square и LifeSquare тестового датасета:\t {sub_square_test}')

Разница медиан Square и LifeSquare тестового датасета:	 19.996253


In [69]:
df_test.loc[df_test['LifeSquare'].isnull() & (df_test['Square'] > sub_square_test), 'LifeSquare'] = df_test['Square'] - lsquare_median_test

Последнее пустое значение приравниваю к Square

In [70]:
df_train.loc[df_train['LifeSquare'].isnull()]

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price
6945,12927,62,2.0,17.414181,NaN,1.0,4,20.0,1977,0.072158,B,B,2,629,1,NaN,0,0,A,180750.471749


In [71]:
df_train.loc[df_train['LifeSquare'].isnull(), 'LifeSquare'] = df_train['Square']

In [72]:
df_train.loc[df_train['LifeSquare'].isnull(), 'LifeSquare'].sum()

0.0

In [73]:
df_test.loc[df_test['LifeSquare'].isnull(), 'LifeSquare'].sum()

0.0

##### Healthcare_1

In [74]:
hc1_median_train = round(df_train["Healthcare_1"].median(), 3)
hc1_median_test = round(df_test["Healthcare_1"].median(), 3)
print(f'Медиана Healthcare_1 обучающего датасета:\t {hc1_median_train}')
print(f'Медиана Healthcare_1 тестового датасета:\t {hc1_median_test}')

Медиана Healthcare_1 обучающего датасета:	 900.0
Медиана Healthcare_1 тестового датасета:	 900.0


In [75]:
df_train.loc[df_train['Healthcare_1'].isnull(), 'Healthcare_1'] = hc1_median_train

In [76]:
df_test.loc[df_test['Healthcare_1'].isnull(), 'Healthcare_1'] = hc1_median_test

In [77]:
df_train.isnull().sum()

Id               0
DistrictId       0
Rooms            0
Square           0
LifeSquare       0
KitchenSquare    0
Floor            0
HouseFloor       0
HouseYear        0
Ecology_1        0
Ecology_2        0
Ecology_3        0
Social_1         0
Social_2         0
Social_3         0
Healthcare_1     0
Helthcare_2      0
Shops_1          0
Shops_2          0
Price            0
dtype: int64

In [78]:
df_test.isnull().sum()

Id               0
DistrictId       0
Rooms            0
Square           0
LifeSquare       0
KitchenSquare    0
Floor            0
HouseFloor       0
HouseYear        0
Ecology_1        0
Ecology_2        0
Ecology_3        0
Social_1         0
Social_2         0
Social_3         0
Healthcare_1     0
Helthcare_2      0
Shops_1          0
Shops_2          0
dtype: int64

### Обработка выбросов обучающего датасета

In [79]:
df_train.describe()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Price
count,10000.00000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,8383.40770,50.400800,1.890500,56.315775,36.130546,6.273300,8.526700,12.609400,3.990166e+03,0.118858,24.687000,5352.157400,8.039200,1026.358900,1.319500,4.231300,214138.857399
std,4859.01902,43.587592,0.839512,21.058732,76.647075,28.560917,5.241148,6.775974,2.005003e+05,0.119025,17.532614,4006.799803,23.831875,746.662828,1.493601,4.806341,92872.293865
min,0.00000,0.000000,0.000000,1.136859,-6.228072,0.000000,1.000000,0.000000,1.910000e+03,0.000000,0.000000,168.000000,0.000000,0.000000,0.000000,0.000000,59174.778028
25%,4169.50000,20.000000,1.000000,41.774881,25.165748,1.000000,4.000000,9.000000,1.974000e+03,0.017647,6.000000,1564.000000,0.000000,830.000000,0.000000,1.000000,153872.633942
50%,8394.50000,36.000000,2.000000,52.513310,32.781260,6.000000,7.000000,13.000000,1.977000e+03,0.075424,25.000000,5285.000000,2.000000,900.000000,1.000000,3.000000,192269.644879
75%,12592.50000,75.000000,2.000000,65.900625,41.427234,9.000000,12.000000,17.000000,2.001000e+03,0.195781,36.000000,7227.000000,5.000000,990.000000,2.000000,6.000000,249135.462171
max,16798.00000,209.000000,19.000000,641.065193,7480.592129,2014.000000,42.000000,117.000000,2.005201e+07,0.521867,74.000000,19083.000000,141.000000,4849.000000,6.000000,23.000000,633233.466570


##### Rooms

In [80]:
df_train.loc[(df_train['Rooms'] > 10) | (df_train['Rooms'] == 0)]

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price
1397,12638,27,0.0,138.427694,136.215499,0.0,4,3.0,2016,0.075424,B,B,11,3097,0,900.0,0,0,B,268394.744389
1454,8491,1,19.0,42.006046,21.779288,7.0,17,17.0,2014,0.007122,B,B,1,264,0,900.0,0,1,B,78364.616704
1981,7917,27,0.0,212.932361,211.231125,0.0,2,3.0,2008,0.211401,B,B,9,1892,0,900.0,0,1,B,302211.260887
2269,7317,27,0.0,41.790881,32.781260,0.0,13,0.0,1977,0.211401,B,B,9,1892,0,900.0,0,1,B,98129.976788
3911,770,28,0.0,49.483501,32.781260,0.0,16,0.0,2015,0.118537,B,B,30,6207,1,1183.0,1,0,B,217009.338463
4366,456,6,0.0,81.491446,32.781260,0.0,4,0.0,1977,0.243205,B,B,5,1564,0,540.0,0,0,B,212864.799112
4853,3224,27,0.0,2.377248,0.873147,0.0,1,0.0,1977,0.017647,B,B,2,469,0,900.0,0,0,B,126596.941798
6149,3159,88,0.0,38.697117,19.345131,9.0,9,16.0,1982,0.127376,B,B,43,8429,3,900.0,3,9,B,158998.110646
8834,9443,27,0.0,87.762616,85.125471,0.0,5,15.0,1977,0.211401,B,B,9,1892,0,900.0,0,1,B,219281.918007


In [81]:
df_train.loc[(df_train['Rooms'] > 10) | (df_train['Rooms'] == 0), 'Rooms'] = df_train['Rooms'].median()

##### Square

In [82]:
averange_square_per_room = round(df_train['Square'].mean() / df_train['Rooms'].mean(), 6)
print(f'Средняя общая площадь квартиры с 1 комнатой:\t{averange_square_per_room}')

Средняя общая площадь квартиры с 1 комнатой:	29.790402


In [83]:
df_train.loc[(df_train['Square'] > 200) | (df_train['Square'] < 15), 'Square'] = df_train['Rooms'] * averange_square_per_room

#### LifeSquare

In [84]:
# Данные значения заполняю разностью Square и sub_square_train
df_train.loc[(df_train['LifeSquare'] > df_train['Square']) | (df_train['LifeSquare'] <= 0), 'LifeSquare'] = df_train['Square'] - sub_square_train

In [85]:
# Также заполняю разностью Square и sub_square_train
df_train.loc[df_train['LifeSquare'] <= 10, 'LifeSquare'] = df_train['Square'] - sub_square_train

In [86]:
# Отрицательным значениям присваиваю среднее значения для квартир с общей площадью до 20 м2
df_train.loc[df_train['LifeSquare'] <= 0, 'LifeSquare'] = df_train.loc[df_train['LifeSquare'] <= 20, 'LifeSquare'].mean()

##### KitchenSquare

In [87]:
df_train.loc[(df_train['KitchenSquare'] > 150) | (df_train['KitchenSquare'] < 5)]

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price
2,4765,53,2.0,44.947953,29.197612,0.0,8,12.0,1968,0.049637,B,B,34,7759,0,229.0,1,3,B,220925.908524
7,11993,74,2.0,80.312926,32.781260,0.0,14,0.0,1977,0.075779,B,B,6,1437,3,900.0,0,2,B,221244.156664
8,5172,1,2.0,64.511437,32.781260,1.0,9,17.0,1977,0.007122,B,B,1,264,0,900.0,0,1,B,229102.795999
12,6452,13,2.0,54.522805,31.759753,1.0,8,12.0,1999,0.090799,B,B,74,19083,2,900.0,5,15,B,229220.372609
14,10953,27,1.0,53.769097,52.408027,1.0,5,4.0,1977,0.072158,B,B,2,629,1,900.0,0,0,A,140647.565937
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9985,16159,27,1.0,38.968338,32.781260,1.0,10,1.0,1977,0.211401,B,B,9,1892,0,900.0,0,1,B,95529.569405
9986,2996,1,2.0,64.994245,32.781260,1.0,9,22.0,1977,0.007122,B,B,1,264,0,900.0,0,1,B,200332.511658
9987,15623,23,2.0,55.620092,32.781260,0.0,2,4.0,1977,0.034656,B,B,0,168,0,900.0,0,0,B,115797.553006
9994,4158,23,2.0,64.864198,32.781260,1.0,17,17.0,1977,0.075779,B,B,6,1437,3,900.0,0,2,B,171323.515653


In [88]:
df_train.loc[(df_train['KitchenSquare'] > 150) | (df_train['KitchenSquare'] <= 5), 'KitchenSquare'] = df_train['KitchenSquare'].median()

##### HouseYear

In [89]:
df_train.loc[df_train['HouseYear'] > 2020]

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price
1497,10814,109,1.0,37.265070,20.239714,9.0,9,12.0,20052011,0.136330,B,B,30,6141,10,262.0,3,6,B,254084.534396
4189,11607,147,2.0,44.791836,28.360393,6.0,4,9.0,4968,0.319809,B,B,25,4756,16,2857.0,5,8,B,243028.603096


In [90]:
df_train.loc[df_train['HouseYear'] == 20052011.0, 'HouseYear'] = 2005

In [91]:
df_train.loc[df_train['HouseYear'] == 4968.0, 'HouseYear'] = 1968

##### HouseFloor

In [92]:
df_train.loc[df_train['HouseFloor'] < df_train['Floor'], 'HouseFloor'] = df_train['HouseFloor'].median() - df_train['Floor'].median()

In [93]:
df_train.describe()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Price
count,10000.00000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,8383.40770,50.400800,1.890400,56.185769,34.964274,7.657200,8.526700,12.575600,1984.865700,0.118858,24.687000,5352.157400,8.039200,1026.358900,1.319500,4.231300,214138.857399
std,4859.01902,43.587592,0.820156,18.870251,14.230940,3.901334,5.241148,6.251399,18.411517,0.119025,17.532614,4006.799803,23.831875,746.662828,1.493601,4.806341,92872.293865
min,0.00000,0.000000,1.000000,16.117154,6.821138,6.000000,1.000000,1.000000,1910.000000,0.000000,0.000000,168.000000,0.000000,0.000000,0.000000,0.000000,59174.778028
25%,4169.50000,20.000000,1.000000,41.783182,24.968112,6.000000,4.000000,6.000000,1974.000000,0.017647,6.000000,1564.000000,0.000000,830.000000,0.000000,1.000000,153872.633942
50%,8394.50000,36.000000,2.000000,52.537977,32.781260,6.000000,7.000000,12.000000,1977.000000,0.075424,25.000000,5285.000000,2.000000,900.000000,1.000000,3.000000,192269.644879
75%,12592.50000,75.000000,2.000000,65.894295,40.689342,9.000000,12.000000,17.000000,2001.000000,0.195781,36.000000,7227.000000,5.000000,990.000000,2.000000,6.000000,249135.462171
max,16798.00000,209.000000,10.000000,198.930182,161.504222,123.000000,42.000000,117.000000,2020.000000,0.521867,74.000000,19083.000000,141.000000,4849.000000,6.000000,23.000000,633233.466570


### Обработка выбросов обучающего датасета

In [94]:
df_test.describe()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.00000,5000.000000
mean,8412.595400,51.279200,1.910000,56.449500,35.433700,5.976800,8.632000,12.601000,1984.392600,0.119874,24.933800,5406.900000,8.262600,1029.396400,1.31940,4.242800
std,4832.674037,44.179466,0.838594,19.092787,16.000098,9.950018,5.483228,6.789213,18.573149,0.120070,17.532202,4026.614773,23.863762,766.595258,1.47994,4.777365
min,1.000000,0.000000,0.000000,1.378543,-2.042510,0.000000,1.000000,0.000000,1908.000000,0.000000,0.000000,168.000000,0.000000,0.000000,0.00000,0.000000
25%,4221.750000,21.000000,1.000000,41.906231,25.441819,1.000000,4.000000,9.000000,1973.000000,0.019509,6.000000,1564.000000,0.000000,810.000000,0.00000,1.000000
50%,8320.500000,37.000000,2.000000,52.921340,32.925087,6.000000,7.000000,12.000000,1977.000000,0.072158,25.000000,5285.000000,2.000000,900.000000,1.00000,3.000000
75%,12598.250000,77.000000,2.000000,66.285129,41.908732,9.000000,12.000000,17.000000,2000.000000,0.195781,36.000000,7287.000000,5.000000,990.000000,2.00000,6.000000
max,16795.000000,212.000000,17.000000,223.453689,303.071094,620.000000,78.000000,99.000000,2020.000000,0.521867,74.000000,19083.000000,141.000000,4849.000000,6.00000,23.000000


##### Rooms

In [95]:
df_test.loc[(df_test['Rooms'] > 10) | (df_test['Rooms'] == 0)]

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2
2406,3343,58,0.0,116.824201,113.692424,0.0,3,3.0,1977,0.437885,B,B,23,5735,3,1084.0,0,5,B
2524,10729,27,0.0,76.345154,42.820796,12.0,14,0.0,1977,0.017647,B,B,2,469,0,900.0,0,0,B
3398,1435,111,17.0,52.866107,32.528342,8.0,15,17.0,1987,0.093443,B,B,23,4635,5,3300.0,2,4,B


In [96]:
df_test.loc[(df_test['Rooms'] > 10) | (df_test['Rooms'] == 0), 'Rooms'] = df_test['Rooms'].median()

##### Square

In [97]:
averange_square_per_room = round(df_test['Square'].mean() / df_test['Rooms'].mean(), 6)
print(f'Средняя общая площадь квартиры с 1 комнатой:\t{averange_square_per_room}')

Средняя общая площадь квартиры с 1 комнатой:	29.588794


In [98]:
df_test.loc[(df_test['Square'] > 200) | (df_test['Square'] < 15), 'Square'] = df_test['Rooms'] * averange_square_per_room

#### LifeSquare

In [99]:
# Данные значения заполняю разностью Square и sub_square_train
df_test.loc[(df_test['LifeSquare'] > df_test['Square']) | (df_test['LifeSquare'] <= 0), 'LifeSquare'] = df_test['Square'] - sub_square_test

In [100]:
# Также заполняю разностью Square и sub_square_train
df_test.loc[df_test['LifeSquare'] <= 10, 'LifeSquare'] = df_test['Square'] - sub_square_test

In [101]:
# Отрицательным значениям присваиваю среднее значения для квартир с общей площадью до 20 м2
df_test.loc[df_test['LifeSquare'] <= 0, 'LifeSquare'] = df_test.loc[df_test['LifeSquare'] <= 20, 'LifeSquare'].mean()

##### KitchenSquare

In [102]:
df_test.loc[(df_test['KitchenSquare'] > 130) | (df_test['KitchenSquare'] <= 4), 'KitchenSquare'] = df_test['KitchenSquare'].median()

##### HouseFloor

In [103]:
df_test.loc[df_test['HouseFloor'] < df_test['Floor'], 'HouseFloor'] = df_test['HouseFloor'].median() - df_test['Floor'].median()

In [104]:
df_test.describe()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1
count,5000.000000,5000.000000,5000.00000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.00000,5000.000000
mean,8412.595400,51.279200,1.90780,56.468159,35.140302,7.465200,8.632000,12.324600,1984.392600,0.119874,24.933800,5406.900000,8.262600,1029.396400,1.31940,4.242800
std,4832.674037,44.179466,0.81008,18.780841,14.163037,3.608849,5.483228,6.469065,18.573149,0.120070,17.532202,4026.614773,23.863762,766.595258,1.47994,4.777365
min,1.000000,0.000000,1.00000,16.319015,6.723470,5.000000,1.000000,1.000000,1908.000000,0.000000,0.000000,168.000000,0.000000,0.000000,0.00000,0.000000
25%,4221.750000,21.000000,1.00000,41.912403,25.176740,6.000000,4.000000,5.000000,1973.000000,0.019509,6.000000,1564.000000,0.000000,810.000000,0.00000,1.000000
50%,8320.500000,37.000000,2.00000,52.924953,32.925087,6.000000,7.000000,12.000000,1977.000000,0.072158,25.000000,5285.000000,2.000000,900.000000,1.00000,3.000000
75%,12598.250000,77.000000,2.00000,66.285129,41.221615,9.000000,12.000000,17.000000,2000.000000,0.195781,36.000000,7287.000000,5.000000,990.000000,2.00000,6.000000
max,16795.000000,212.000000,6.00000,189.679576,148.732782,112.000000,78.000000,99.000000,2020.000000,0.521867,74.000000,19083.000000,141.000000,4849.000000,6.00000,23.000000


### Обрабатываю object-признаки в обучающем датасете

In [105]:
df_train.dtypes

Id                 int64
DistrictId         int64
Rooms            float64
Square           float64
LifeSquare       float64
KitchenSquare    float64
Floor              int64
HouseFloor       float64
HouseYear          int64
Ecology_1        float64
Ecology_2         object
Ecology_3         object
Social_1           int64
Social_2           int64
Social_3           int64
Healthcare_1     float64
Helthcare_2        int64
Shops_1            int64
Shops_2           object
Price            float64
dtype: object

In [106]:
df_train['Ecology_2'].value_counts()

B    9903
A      97
Name: Ecology_2, dtype: int64

In [107]:
df_train['Ecology_3'].value_counts()

B    9725
A     275
Name: Ecology_3, dtype: int64

In [108]:
df_train['Shops_2'].value_counts()

B    9175
A     825
Name: Shops_2, dtype: int64

Заменяю категориальные признаки бинарными данными

In [109]:
df_train['Ecology_2_bin'] = df_train['Ecology_2'].replace({'A':0, 'B':1})
df_train['Ecology_3_bin'] = df_train['Ecology_3'].replace({'A':0, 'B':1})
df_train['Shops_2_bin'] = df_train['Shops_2'].replace({'A':0, 'B':1})

Изменяю тип признака `Id` на `str`

In [110]:
df_train['Id'] = df_train['Id'].astype(str)

In [111]:
df_train.head()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price,Ecology_2_bin,Ecology_3_bin,Shops_2_bin
0,14038,35,2.0,47.981561,29.442751,6.0,7,9.0,1969,0.089040,B,B,33,7976,5,900.0,0,11,B,184966.930730,1,1,1
1,15053,41,3.0,65.683640,40.049543,8.0,7,9.0,1978,0.000070,B,B,46,10309,1,240.0,1,16,B,300009.450063,1,1,1
2,4765,53,2.0,44.947953,29.197612,6.0,8,12.0,1968,0.049637,B,B,34,7759,0,229.0,1,3,B,220925.908524,1,1,1
3,5809,58,2.0,53.352981,52.731512,9.0,8,17.0,1977,0.437885,B,B,23,5735,3,1084.0,0,5,B,175616.227217,1,1,1
4,10783,99,1.0,39.649192,23.776169,7.0,11,12.0,1976,0.012339,B,B,35,5776,1,2078.0,2,4,B,150226.531644,1,1,1


In [112]:
#obj_columns = ['Ecology_2', 'Ecology_3', 'Shops_2']

In [113]:
#train_dummies = pd.get_dummies(df_train[obj_columns])
#train_dummies

In [114]:
#df_train = pd.concat([df_train, train_dummies], axis=1)
#df_train.head()

### Обрабатываю object-признаки в тестовом датасете

In [115]:
df_test.dtypes

Id                 int64
DistrictId         int64
Rooms            float64
Square           float64
LifeSquare       float64
KitchenSquare    float64
Floor              int64
HouseFloor       float64
HouseYear          int64
Ecology_1        float64
Ecology_2         object
Ecology_3         object
Social_1           int64
Social_2           int64
Social_3           int64
Healthcare_1     float64
Helthcare_2        int64
Shops_1            int64
Shops_2           object
dtype: object

Заменяю категориальные признаки бинарными

In [116]:
df_test['Ecology_2_bin'] = df_test['Ecology_2'].replace({'A':0, 'B':1})
df_test['Ecology_3_bin'] = df_test['Ecology_3'].replace({'A':0, 'B':1})
df_test['Shops_2_bin'] = df_test['Shops_2'].replace({'A':0, 'B':1})

Изменяю тип признака `Id` на `str`

In [117]:
df_train['Id'] = df_train['Id'].astype(str)

In [118]:
df_test.head()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Ecology_2_bin,Ecology_3_bin,Shops_2_bin
0,725,58,2.0,49.882643,33.432782,6.0,6,14.0,1972,0.310199,B,B,11,2748,1,900.0,0,0,B,1,1,1
1,15856,74,2.0,69.263183,32.925087,6.0,6,5.0,1977,0.075779,B,B,6,1437,3,900.0,0,2,B,1,1,1
2,5480,190,1.0,29.588794,15.948246,12.0,2,5.0,1909,0.000000,B,B,30,7538,87,4702.0,5,5,B,1,1,1
3,15664,47,2.0,73.046609,51.940842,9.0,22,22.0,2007,0.101872,B,B,23,4583,3,900.0,3,3,B,1,1,1
4,14275,27,1.0,47.527111,43.387569,6.0,17,17.0,2017,0.072158,B,B,2,629,1,900.0,0,0,A,1,1,0


In [ ]:
#test_dummies = pd.get_dummies(df_test[obj_columns])
#test_dummies

In [ ]:
#df_test = pd.concat([df_test, test_dummies], axis=1)
#df_test.head()

### DistrictSize, IsDistrictLarge

##### TRAIN

In [119]:
df_train['DistrictId'].value_counts()

27     851
1      652
23     565
6      511
9      294
      ... 
199      1
205      1
174      1
209      1
207      1
Name: DistrictId, Length: 205, dtype: int64

In [120]:
district_size_train = df_train['DistrictId'].value_counts().reset_index()\
               .rename(columns={'index':'DistrictId', 'DistrictId':'DistrictSize'})

district_size_train.head()

,DistrictId,DistrictSize
0,27,851
1,1,652
2,23,565
3,6,511
4,9,294


In [121]:
df_train = df_train.merge(district_size_train, on='DistrictId', how='left')
df_train.head()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price,Ecology_2_bin,Ecology_3_bin,Shops_2_bin,DistrictSize
0,14038,35,2.0,47.981561,29.442751,6.0,7,9.0,1969,0.089040,B,B,33,7976,5,900.0,0,11,B,184966.930730,1,1,1,22
1,15053,41,3.0,65.683640,40.049543,8.0,7,9.0,1978,0.000070,B,B,46,10309,1,240.0,1,16,B,300009.450063,1,1,1,87
2,4765,53,2.0,44.947953,29.197612,6.0,8,12.0,1968,0.049637,B,B,34,7759,0,229.0,1,3,B,220925.908524,1,1,1,174
3,5809,58,2.0,53.352981,52.731512,9.0,8,17.0,1977,0.437885,B,B,23,5735,3,1084.0,0,5,B,175616.227217,1,1,1,179
4,10783,99,1.0,39.649192,23.776169,7.0,11,12.0,1976,0.012339,B,B,35,5776,1,2078.0,2,4,B,150226.531644,1,1,1,97


In [122]:
(df_train['DistrictSize'] > 100).value_counts()

True     5069
False    4931
Name: DistrictSize, dtype: int64

In [123]:
df_train['IsDistrictLarge'] = (df_train['DistrictSize'] > 100).astype(int)

In [124]:
df_train.head()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price,Ecology_2_bin,Ecology_3_bin,Shops_2_bin,DistrictSize,IsDistrictLarge
0,14038,35,2.0,47.981561,29.442751,6.0,7,9.0,1969,0.089040,B,B,33,7976,5,900.0,0,11,B,184966.930730,1,1,1,22,0
1,15053,41,3.0,65.683640,40.049543,8.0,7,9.0,1978,0.000070,B,B,46,10309,1,240.0,1,16,B,300009.450063,1,1,1,87,0
2,4765,53,2.0,44.947953,29.197612,6.0,8,12.0,1968,0.049637,B,B,34,7759,0,229.0,1,3,B,220925.908524,1,1,1,174,1
3,5809,58,2.0,53.352981,52.731512,9.0,8,17.0,1977,0.437885,B,B,23,5735,3,1084.0,0,5,B,175616.227217,1,1,1,179,1
4,10783,99,1.0,39.649192,23.776169,7.0,11,12.0,1976,0.012339,B,B,35,5776,1,2078.0,2,4,B,150226.531644,1,1,1,97,0


##### TEST

In [125]:
df_test['DistrictId'].value_counts()

27     391
1      344
23     264
6      257
9      132
      ... 
212      1
208      1
204      1
200      1
193      1
Name: DistrictId, Length: 201, dtype: int64

In [126]:
district_size_test = df_test['DistrictId'].value_counts().reset_index()\
               .rename(columns={'index':'DistrictId', 'DistrictId':'DistrictSize'})

district_size_test.head()

,DistrictId,DistrictSize
0,27,391
1,1,344
2,23,264
3,6,257
4,9,132


In [127]:
df_test = df_test.merge(district_size_test, on='DistrictId', how='left')
df_test.head()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Ecology_2_bin,Ecology_3_bin,Shops_2_bin,DistrictSize
0,725,58,2.0,49.882643,33.432782,6.0,6,14.0,1972,0.310199,B,B,11,2748,1,900.0,0,0,B,1,1,1,91
1,15856,74,2.0,69.263183,32.925087,6.0,6,5.0,1977,0.075779,B,B,6,1437,3,900.0,0,2,B,1,1,1,61
2,5480,190,1.0,29.588794,15.948246,12.0,2,5.0,1909,0.000000,B,B,30,7538,87,4702.0,5,5,B,1,1,1,2
3,15664,47,2.0,73.046609,51.940842,9.0,22,22.0,2007,0.101872,B,B,23,4583,3,900.0,3,3,B,1,1,1,49
4,14275,27,1.0,47.527111,43.387569,6.0,17,17.0,2017,0.072158,B,B,2,629,1,900.0,0,0,A,1,1,0,391


In [128]:
df_test.rename(columns={'DistrictSize_y': 'DistrictSize'}, inplace=True)

In [129]:
(df_test['DistrictSize'] > 100).value_counts()

False    3375
True     1625
Name: DistrictSize, dtype: int64

In [130]:
df_test['IsDistrictLarge'] = (df_test['DistrictSize'] > 100).astype(int)

In [131]:
df_test.head()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Ecology_2_bin,Ecology_3_bin,Shops_2_bin,DistrictSize,IsDistrictLarge
0,725,58,2.0,49.882643,33.432782,6.0,6,14.0,1972,0.310199,B,B,11,2748,1,900.0,0,0,B,1,1,1,91,0
1,15856,74,2.0,69.263183,32.925087,6.0,6,5.0,1977,0.075779,B,B,6,1437,3,900.0,0,2,B,1,1,1,61,0
2,5480,190,1.0,29.588794,15.948246,12.0,2,5.0,1909,0.000000,B,B,30,7538,87,4702.0,5,5,B,1,1,1,2,0
3,15664,47,2.0,73.046609,51.940842,9.0,22,22.0,2007,0.101872,B,B,23,4583,3,900.0,3,3,B,1,1,1,49,0
4,14275,27,1.0,47.527111,43.387569,6.0,17,17.0,2017,0.072158,B,B,2,629,1,900.0,0,0,A,1,1,0,391,1


### MedPriceByDistrict

##### TRAIN

In [136]:
med_price_by_district = df_train.groupby(['DistrictId', 'Rooms'], as_index=False).agg({'Price':'median'})\
                       .rename(columns={'Price':'MedPriceByDistrict'})

med_price_by_district.head()

,DistrictId,Rooms,MedPriceByDistrict
0,0,1.0,142799.434052
1,0,2.0,176210.812334
2,0,3.0,300610.359425
3,1,1.0,146735.671740
4,1,2.0,202666.310126


In [133]:
df_train = df_train.merge(med_price_by_district, on=['DistrictId', 'Rooms'], how='left')
df_train.head()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price,Ecology_2_bin,Ecology_3_bin,Shops_2_bin,DistrictSize,IsDistrictLarge,MedPriceByDistrict
0,14038,35,2.0,47.981561,29.442751,6.0,7,9.0,1969,0.089040,B,B,33,7976,5,900.0,0,11,B,184966.930730,1,1,1,22,0,207786.135836
1,15053,41,3.0,65.683640,40.049543,8.0,7,9.0,1978,0.000070,B,B,46,10309,1,240.0,1,16,B,300009.450063,1,1,1,87,0,300009.450063
2,4765,53,2.0,44.947953,29.197612,6.0,8,12.0,1968,0.049637,B,B,34,7759,0,229.0,1,3,B,220925.908524,1,1,1,174,1,263206.223152
3,5809,58,2.0,53.352981,52.731512,9.0,8,17.0,1977,0.437885,B,B,23,5735,3,1084.0,0,5,B,175616.227217,1,1,1,179,1,166809.665642
4,10783,99,1.0,39.649192,23.776169,7.0,11,12.0,1976,0.012339,B,B,35,5776,1,2078.0,2,4,B,150226.531644,1,1,1,97,0,158539.052715


In [134]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 0 to 9999
Data columns (total 26 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Id                  10000 non-null  object 
 1   DistrictId          10000 non-null  int64  
 2   Rooms               10000 non-null  float64
 3   Square              10000 non-null  float64
 4   LifeSquare          10000 non-null  float64
 5   KitchenSquare       10000 non-null  float64
 6   Floor               10000 non-null  int64  
 7   HouseFloor          10000 non-null  float64
 8   HouseYear           10000 non-null  int64  
 9   Ecology_1           10000 non-null  float64
 10  Ecology_2           10000 non-null  object 
 11  Ecology_3           10000 non-null  object 
 12  Social_1            10000 non-null  int64  
 13  Social_2            10000 non-null  int64  
 14  Social_3            10000 non-null  int64  
 15  Healthcare_1        10000 non-null  float64
 16  Helth

##### TEST

In [137]:
df_test = df_test.merge(med_price_by_district, on=['DistrictId', 'Rooms'], how='left')
df_test.head()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Ecology_2_bin,Ecology_3_bin,Shops_2_bin,DistrictSize,IsDistrictLarge,MedPriceByDistrict
0,725,58,2.0,49.882643,33.432782,6.0,6,14.0,1972,0.310199,B,B,11,2748,1,900.0,0,0,B,1,1,1,91,0,166809.665642
1,15856,74,2.0,69.263183,32.925087,6.0,6,5.0,1977,0.075779,B,B,6,1437,3,900.0,0,2,B,1,1,1,61,0,243031.695525
2,5480,190,1.0,29.588794,15.948246,12.0,2,5.0,1909,0.000000,B,B,30,7538,87,4702.0,5,5,B,1,1,1,2,0,NaN
3,15664,47,2.0,73.046609,51.940842,9.0,22,22.0,2007,0.101872,B,B,23,4583,3,900.0,3,3,B,1,1,1,49,0,222227.584211
4,14275,27,1.0,47.527111,43.387569,6.0,17,17.0,2017,0.072158,B,B,2,629,1,900.0,0,0,A,1,1,0,391,1,118327.090620


In [138]:
df_test.loc[df_test['MedPriceByDistrict'].isnull()]

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Ecology_2_bin,Ecology_3_bin,Shops_2_bin,DistrictSize,IsDistrictLarge,MedPriceByDistrict
2,5480,190,1.0,29.588794,15.948246,12.0,2,5.0,1909,0.000000e+00,B,B,30,7538,87,4702.0,5,5,B,1,1,1,2,0,NaN
51,15748,212,2.0,75.655182,36.877626,15.0,2,7.0,1929,0.000000e+00,B,B,8,1423,37,562.0,1,8,B,1,1,1,1,0,NaN
149,3764,193,4.0,97.931642,72.386905,8.0,5,5.0,1957,3.198094e-01,B,B,25,4756,16,2857.0,5,8,B,1,1,1,1,0,NaN
255,10816,51,3.0,97.889332,58.640564,12.0,4,5.0,1929,1.501689e-02,B,B,17,3079,59,2890.0,0,8,B,1,1,1,8,0,NaN
263,11407,79,4.0,37.920348,21.357487,6.0,6,17.0,2014,3.074667e-01,B,A,30,5048,9,325.0,2,5,B,1,0,1,24,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4643,2196,52,4.0,104.638510,62.185444,10.0,4,22.0,2005,3.711490e-01,B,B,34,7065,1,750.0,2,5,B,1,1,1,67,0,NaN
4711,429,92,4.0,107.747554,60.377351,13.0,10,17.0,2008,4.605564e-01,B,B,20,4386,14,900.0,1,5,B,1,1,1,7,0,NaN
4719,13426,209,3.0,57.320443,43.187837,6.0,4,5.0,1967,4.024276e-02,B,B,25,3478,11,2729.0,3,1,B,1,1,1,2,0,NaN
4725,15237,115,1.0,31.605210,18.247347,5.0,3,5.0,1962,1.800000e-09,B,B,16,3681,73,1815.0,2,5,B,1,1,1,6,0,NaN


In [139]:
for i in df_test.index[df_test['MedPriceByDistrict'].isnull()]:
    district_size = df_test.iloc[i, 22]
    df_test.iat[i, 24] = df_test.loc[(df_test['DistrictSize'] == district_size) & (~df_test['MedPriceByDistrict'].isnull()), 'MedPriceByDistrict'].median()

In [143]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5000 entries, 0 to 4999
Data columns (total 25 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Id                  5000 non-null   int64  
 1   DistrictId          5000 non-null   int64  
 2   Rooms               5000 non-null   float64
 3   Square              5000 non-null   float64
 4   LifeSquare          5000 non-null   float64
 5   KitchenSquare       5000 non-null   float64
 6   Floor               5000 non-null   int64  
 7   HouseFloor          5000 non-null   float64
 8   HouseYear           5000 non-null   int64  
 9   Ecology_1           5000 non-null   float64
 10  Ecology_2           5000 non-null   object 
 11  Ecology_3           5000 non-null   object 
 12  Social_1            5000 non-null   int64  
 13  Social_2            5000 non-null   int64  
 14  Social_3            5000 non-null   int64  
 15  Healthcare_1        5000 non-null   float64
 16  Helthc

# Удаляю object-признаки в тестовом датасете

In [ ]:
df_test = df_test.loc[:, (df_test.columns != 'Ecology_2') & (df_test.columns != 'Ecology_3') & (df_test.columns != 'Shops_2')]

In [ ]:
df_test.head()

### Сохраняю подготовленные датасеты

In [ ]:
df_train.to_csv(PREPARED_TRAIN_DATASET_PATH, index=False, encoding='utf-8')

In [ ]:
df_test.to_csv(PREPARED_TEST_DATASET_PATH, index=False, encoding='utf-8')

In [ ]:
df_train = pd.read_csv(PREPARED_TRAIN_DATASET_PATH)

In [ ]:
df_train.head()

In [ ]:
df_train.shape

### Визуализация обучающего датасета

In [ ]:
title_font = {
    "fontsize": 16,
    "fontweight": "bold",
    "color": "darkgrey",
    "family": "arial",
}

label_font = {
    "fontsize": 10,
    "family": "arial",
}

In [ ]:
plt.figure(figsize=(6,4))
df_train['Price'].hist()  

plt.title('Price histogramm', fontdict=title_font)
plt.ylabel('count', fontdict=label_font)
plt.xlabel('Price', fontdict=label_font)

plt.show()

In [ ]:
plt.figure(figsize = (13,10))

sns.set(font_scale=0.5)
sns.heatmap(df_train.corr(), annot=True, linewidths=.5, cmap='GnBu')

plt.title('Correlation matrix', fontdict=title_font)

plt.show()

In [ ]:
plt.figure(figsize=(10,10))

sns.scatterplot(x=df_train['Price'], y=df_train['Square'])

plt.title('Square & Price', fontdict=title_font)
plt.ylabel('Square', fontdict=label_font)
plt.xlabel('Price', fontdict=label_font)

plt.show()

### Стандартизирую признаки

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
df_train.dtypes

In [ ]:
columns_for_stand = df_train.columns[:19]

In [ ]:
columns_for_stand = df_train[columns_for_stand].select_dtypes(include=['float64', 'int64']).columns.tolist()
columns_for_stand.append('DistrictSize')
columns_for_stand.append('MedPriceByDistrict')
columns_for_stand

In [ ]:
scaler = StandardScaler()
standard_features = scaler.fit_transform(df_train[columns_for_stand])

In [ ]:
df_train[columns_for_stand] = pd.DataFrame(standard_features, columns=columns_for_stand)

In [ ]:
df_train.head()

In [ ]:
df_train.std()

##### Удаляю object-признаки

In [ ]:
feature_names = df_train.columns
feature_names

In [ ]:
df_train.info()

In [ ]:
feature_names = df_train[feature_names].select_dtypes(include=['float64', 'int64', 'uint8']).columns.tolist()
feature_names

In [ ]:
df = df_train[feature_names]
df.head()

In [ ]:
# сохраняю стандартизированный датасет
df.to_csv(SCALED_TRAIN_DATASET_PATH, index=False, encoding='utf-8')

### Разбиваю обучающи датасет на train и valid

In [ ]:
df = pd.read_csv(SCALED_TRAIN_DATASET_PATH)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X = df.loc[:, df.columns != 'Price']
y = df.loc[:, df.columns == 'Price']

In [ ]:
X.head()

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, shuffle=True, random_state=27)

##### Сохраняю датасеты в файлы

In [ ]:
X_train.to_csv(X_TRAIN, index=False, encoding='utf-8')
X_valid.to_csv(X_VALID, index=False, encoding='utf-8')
y_train.to_csv(Y_TRAIN, index=False, encoding='utf-8')
y_valid.to_csv(Y_VALID, index=False, encoding='utf-8')

In [6]:
X_train = pd.read_csv(X_TRAIN)
X_valid = pd.read_csv(X_VALID)
y_train = pd.read_csv(Y_TRAIN)
y_valid = pd.read_csv(Y_VALID)

In [ ]:
print(X_train.shape)
print(y_train.shape)
print(X_valid.shape)
print(y_valid.shape)

In [ ]:
from sklearn.metrics import r2_score as r2

In [ ]:
def r2_score(true_values, pred_values):
    print("R2:\t" + str(round(r2(true_values, pred_values), 3)) + "\n")
    
    plt.figure(figsize=(10,10))
    
    plt.scatter(pred_values, true_values)
   
    plt.title('True vs Predicted values', fontdict=title_font)
    plt.xlabel('Predicted values', fontdict=label_font)
    plt.ylabel('True values', fontdict=label_font)
        
    plt.show()
    return None

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

### LinearRegression

In [ ]:
lr_model = LinearRegression()

In [ ]:
lr_model.fit(X_train, y_train)

In [ ]:
y_train_preds = lr_model.predict(X_train)
r2(y_train, y_train_preds)

In [ ]:
y_valid_preds = lr_model.predict(X_valid)
r2_score(y_valid, y_valid_preds)

### Random Forrest

In [ ]:
rf_model = RandomForestRegressor()
rf_model.fit(X_train, y_train)

In [ ]:
y_train_preds = rf_model.predict(X_train)
r2(y_train, y_train_preds)

In [ ]:
y_valid_preds = rf_model.predict(X_valid)
r2_score(y_valid, y_valid_preds)

### Gradient Boosting

In [ ]:
gb_model = GradientBoostingRegressor()
gb_model.fit(X_train, y_train)

In [ ]:
y_train_preds = gb_model.predict(X_train)
r2(y_train, y_train_preds)

In [ ]:
y_valid_preds = gb_model.predict(X_valid)
r2_score(y_valid, y_valid_preds)

### Подбор параметров RandomForestRegressor

In [ ]:
rf_model = RandomForestRegressor(random_state=27)
rf_model

In [ ]:
params = {'n_estimators':[100, 200, 300],
          'max_features': [5, 7, 9, 11, 13],
          'max_depth':[5, 7, 9, 11, 13]}

gs_rf = GridSearchCV(rf_model, params, scoring='r2', cv=KFold(n_splits=5, random_state=27, shuffle=True), n_jobs=-1)
gs_rf.fit(X, y)

In [ ]:
gs_rf.best_params_

In [ ]:
gs_rf.best_score_

### Подбор параметров GradientBoostingRegressor

In [ ]:
gb_model = GradientBoostingRegressor(random_state=27)
gb_model

In [ ]:
params = {'n_estimators':[50, 100, 200, 400, 600], 
          'max_depth':[3, 5, 7, 10, 13]}

gs_gb = GridSearchCV(gb_model, params, scoring='r2', cv=KFold(n_splits=5, random_state=27, shuffle=True), n_jobs=-1)
gs_gb.fit(X, y)

In [ ]:
gs_gb.best_params_

In [ ]:
gs_gb.best_score_

##### Обучение финальной модели

In [ ]:
final_model = GradientBoostingRegressor(n_estimators=200, max_depth=5, random_state=27)
final_model.fit(X_train, y_train)

In [ ]:
y_train_preds = final_model.predict(X_train)
r2(y_train, y_train_preds)

In [ ]:
y_valid_preds = final_model.predict(X_valid)
r2_score(y_valid, y_valid_preds)

##### Важность признаков

In [ ]:
feature_importances = pd.DataFrame(zip(X_train.columns, final_model.feature_importances_), 
                                   columns=['feature_name', 'importance'])

feature_importances.sort_values(by='importance', ascending=False)

In [ ]:
y_valid_preds[:10]

### Сохранение модели

In [ ]:
with open(SCALER_FILE_PATH, 'wb') as file:
    pickle.dump(scaler, file)

In [ ]:
with open(MODEL_PATH, 'wb') as file:
    pickle.dump(final_model, file)

### Предсказание на тестовом датасете

In [ ]:
df_test = pd.read_csv(PREPARED_TEST_DATASET_PATH)

In [ ]:
df_test.head()

In [ ]:
df_test.describe()

In [ ]:
predict_price = pd.DataFrame(df_test['Id'])

In [ ]:
predict_price.head()

In [ ]:
standard_features_test = scaler.fit_transform(df_test[columns_for_stand])

In [ ]:
df_test[columns_for_stand] = pd.DataFrame(standard_features_test, columns=columns_for_stand)

In [ ]:
df_test.head()

In [ ]:
test_preds = final_model.predict(df_test)

In [ ]:
predict_price['Price'] = test_preds

In [ ]:
predict_price

In [ ]:
predict_price.to_csv(PREDICT_PRICE, index=False, encoding='utf-8')

# Кластеризация